# 📊 Modelagem Epidemiológica da COVID-19 no Pará com o Modelo SIRC

Neste notebook é apresentada a aplicação de um **modelo epidemiológico compartimental do tipo SIRC** — *Suscetíveis (S)*, *Infectados (I)*, *Recuperados (R)* e *Casos acumulados (C)* — para descrever e ajustar a dinâmica da pandemia de **COVID-19 no estado do Pará**, com base em dados reais de casos confirmados.

O objetivo central é **ajustar um sistema de equações diferenciais ordinárias (EDOs)** aos dados observados de casos acumulados, estimando parâmetros epidemiológicos relevantes por meio de **otimização não linear**.

---

## 🔬 Visão Geral da Metodologia

O fluxo de trabalho adotado neste estudo pode ser dividido em quatro etapas principais:

### 1️⃣ Leitura e organização dos dados epidemiológicos

Os dados são carregados a partir de um arquivo no formato `parquet`, contendo registros diários de casos confirmados de COVID-19. Neste estudo os dados selecionados são para Belém-Pa. A variável observada principal é o número de **casos novos diários**, a partir do qual se constrói a série de **casos acumulados**.

---

### 2️⃣ Construção das séries temporais

Seja \( C(t) \) o número acumulado de casos no tempo \( t \). A partir da série de casos novos \( \Delta C(t) \), calcula-se:

$$[
C(t) = \sum_{i=1}^{t} \Delta C(i)
]$$

A análise é restrita aos **primeiros 150 dias da pandemia**, período no qual a dinâmica de crescimento é mais informativa para a estimação dos parâmetros epidemiológicos.

---

### 3️⃣ Modelo Epidemiológico SIRC

O modelo SIRC descreve a evolução temporal das populações por meio do seguinte sistema de equações diferenciais ordinárias:


\begin{aligned}
\frac{dS}{dt} &= -\beta \frac{S I}{N} \\
\frac{dI}{dt} &= \beta \frac{S I}{N} - \gamma I \\
\frac{dR}{dt} &= \gamma I \\
\frac{dC}{dt} &= \beta \frac{S I}{N}
\end{aligned}


onde:

- \( S(t) \): número de indivíduos suscetíveis
- \( I(t) \): número de indivíduos infectados
- \( R(t) \): número de indivíduos recuperados
- \( C(t) \): número de casos acumulados
- \( \beta \): taxa de contágio
- \( \gamma \): taxa de recuperação
- \( N \): população total considerada (constante)

A condição inicial do sistema é dada por:


\begin{aligned}
S(0) &= N - I_0 \\
I(0) &= I_0 \\
R(0) &= 0 \\
C(0) &= I_0
\end{aligned}


---

### 4️⃣ Resolução Numérica e Ajuste do Modelo

O sistema de EDOs é resolvido numericamente por meio do método de **integração de Runge–Kutta**, implementado pela função `odeint` da biblioteca `scipy`.

O ajuste do modelo aos dados observados é realizado utilizando a biblioteca **`lmfit`**, que permite:

* Definir parâmetros livres e fixos;
* Impor limites físicos aos parâmetros;
* Estimar incertezas associadas aos parâmetros;
* Avaliar correlações entre eles.

O processo de ajuste minimiza a função objetivo:

$
\min_{\beta, \gamma} \sum_{t} \left[ C_{\text{obs}}(t) - C_{\text{mod}}(t; \beta, \gamma) \right]^2
$

---

## 📈 Métricas de Avaliação

A qualidade do ajuste é avaliada a partir de métricas estatísticas como:

* coeficiente de determinação \( R^2 \);
* qui-quadrado reduzido ( $\chi^2_{\text{red}}$ \);
* Critério de Informação de Akaike (AIC);
* Critério de Informação Bayesiano (BIC).

Essas métricas permitem avaliar tanto a aderência do modelo aos dados quanto a complexidade do ajuste.

---

## 🎯 Importância do Estudo

A modelagem epidemiológica baseada em modelos compartimentais permite:

* Compreender a dinâmica temporal da pandemia em escala regional;
* Estimar parâmetros epidemiológicos relevantes;
* Comparar diferentes regiões e períodos;
* Avaliar a adequação de modelos mecanísticos a dados reais;
* Fornecer subsídios quantitativos para análises retrospectivas e estudos em saúde pública.

Este notebook tem caráter **exploratório e quantitativo**, servindo como base para extensões futuras, como a inclusão de efeitos temporais, intervenções não farmacológicas ou variação de parâmetros no tempo.

## 📌 Resumo dos Resultados

O modelo SIRC explica aproximadamente **97% da variabilidade observada** ($\(R^2 = 0{,}97\)$). Os parâmetros estimados foram **β = 0,514 ± 0,008** e **γ = 0,396 ± 0,008**, com baixas incertezas relativas, indicando boa identificação estatística no período analisado. O número básico de reprodução estimado foi **\($R_0 \approx 1{,}30\$)**, consistente com um cenário de crescimento epidêmico moderado. Observou-se ainda **forte correlação entre β e γ** (0,997), sugerindo que a dinâmica do modelo é majoritariamente controlada pela razão \(β/γ\).



## 🧩 Bibliotecas Utilizadas

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import lmfit
import matplotlib.pyplot as plt
import modelos_epidemiologicos as model
from datetime import datetime

print(datetime.now().isoformat(timespec='seconds'))

## 📥 Leitura e Pré-processamento dos Dados


In [ ]:
# leitura dados
df0= pd.read_parquet("/home/akel/PycharmProjects/Endemic_model/data/caso_full.parquet")
df0['date']=pd.to_datetime(df0['date'],format='%Y-%m-%d')

dfE=df0.groupby(['date', 'state'])['new_confirmed'].sum().reset_index() #agrupar por estados
dfPA= dfE[dfE['state']=='PA' ]                                          #filtrar estado
dfbel= df0[(df0["city"] == "Belém") & (df0['state']=='PA') ]            #filtrar cidade
print('arquivos lidos')
print(datetime.now().isoformat(timespec='seconds'))


## 📈 Visualização dos Dados


In [ ]:
# Visualização
NCP=dfPA['new_confirmed'].values
NCB=dfbel['new_confirmed'].values

# casos acumulados
Cp=np.cumsum(NCP)
Cb=np.cumsum(NCB)

t_date=dfPA['date'].values

# periodo -01/02/2020 - 12/07/2020 (318)
yc=NCP[0:150]
y=Cp[0:150]
x=t_date[0:150] 

plt.figure(figsize=(12, 6))
plt.bar(x, y, color='red', label='Dados originais')
plt.title('Casos aculumados COVID-19 estado do Pará\n 2020-2021', fontsize=14)
plt.xlabel('Data', fontsize=12)
plt.tight_layout()
plt.ylim([0,300000])
plt.xlim(pd.to_datetime(['2020-03-18', '2020-07-30'])) 
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45);  # Rotaciona as datas para melhor legibilidade

## 🧮 Definição do Modelo SIRC


In [ ]:
#funcoes
def solve_sirc(t,N,beta,gamma,I0):
    """Resolve o modelo SIRC e retorna casos acumulados."""
    R0 = 0
    C0 = I0
    S0 = N - I0 - R0
    y0 = [S0, I0, R0, C0]
    sol = odeint(model.SIRC, y0, t, args=(N, beta, gamma))
    return sol[:, 3]

print(datetime.now().isoformat(timespec='seconds'))

## 🎯 Estimativa Inicial dos Parâmetros


In [ ]:
c = y                  
t = np.arange(len(c))
#I0=1.728745
#N=700000
# gamma_i=0.291505
# beta_i=0.437257

R0 = 0
#C0 = I0             # Casos acumulados iniciais = infectados iniciais
#S0 = N  - I0 - R0
#y0 = [S0, I0, R0, C0]


# Estimar chutes inicias e N
K,r,A = model.initial_SIRC(c)

I0 = K / (A + 1)         #infectados inicias
N = 2 * K                #populacao
gamma_i = 2 * r          #taxa de recuperacao
beta_i = 1.5 * gamma_i   #taxa de contagio

# ---------------------------
# PARÂMETROS INICIAIS
# ---------------------------
print('\n' + '='*40)
print('PARÂMETROS INICIAIS'.center(40))
print('='*40)
print(f'{"I0:":<10}{I0:>30.6f}')
print(f'{"N:":<10}{N:>30.6f}')
print(f'{"gamma_i:":<10}{gamma_i:>30.6f}')
print(f'{"beta_i:":<10}{beta_i:>30.6f}')
print(datetime.now().isoformat(timespec='seconds'))

## ⚙️ Ajuste dos Parâmetros com lmfit

In [ ]:
# Configurar modelo
SIRC_model= lmfit.Model(solve_sirc,independent_vars=['t'])


print(N,beta_i,gamma_i,I0)
# # Definição dos limites dos parametros
params = SIRC_model.make_params()
params['N'].set(value=N, vary=False)
params['beta'].set(value=beta_i, min=0.001, max=3.0)
params['gamma'].set(value=gamma_i, min=0.001, max=2.0)
params['I0'].set(value=I0, vary=False)
                # min=1, max=N)
result = SIRC_model.fit(y, params, t=t)
print(result.fit_report())

## 📉 Resultados dos ajustes

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(x,y,'o',label='dados')
plt.plot(x,result.best_fit,label='melhor ajuste');  #melhor ajuste do modelo
plt.fill_between(
    x, 
    result.best_fit - result.residual.std(), 
    result.best_fit+ result.residual.std(), 
    color='Orange', alpha=0.2, label='Incerteza'
);


## 📊 Análise dos Resultados do Ajuste do Modelo SIRC

Os resultados apresentados correspondem ao ajuste do modelo SIRC aos **casos acumulados de COVID-19 no estado do Pará**, considerando um período inicial de 150 dias da pandemia. A seguir, discute-se a qualidade do ajuste, a interpretação dos parâmetros estimados e suas implicações epidemiológicas.

---

### 🔍 Qualidade do Ajuste

O coeficiente de determinação obtido foi:


$R^2 = 0.969$

Esse valor indica que o modelo é capaz de explicar aproximadamente **97% da variabilidade observada** nos dados de casos acumulados, evidenciando uma **excelente aderência global** do modelo SIRC aos dados nesse intervalo temporal. Esse resultado sugere que a estrutura mecanística do modelo é adequada para capturar a dinâmica média de crescimento da pandemia no período analisado.

Por outro lado, o **qui-quadrado reduzido elevado** (( $\chi^2_{\text{red}}$ $\sim$ 4.8 $\times$ 10^8 )) está associado à escala absoluta dos dados (casos acumulados), que envolve números elevados. Como não foi considerada uma modelagem explícita das incertezas observacionais, esse valor não deve ser interpretado isoladamente como indicação de mau ajuste, mas sim como consequência da magnitude da variável-alvo.

Os critérios de informação:


$\text{AIC} = 2998.9, \quad \text{BIC} = 3004.9$


refletem um bom compromisso entre **qualidade do ajuste e complexidade do modelo**, especialmente considerando que apenas **dois parâmetros livres** foram estimados (( \beta ) e ( \gamma )).

---

### 🧮 Interpretação dos Parâmetros Estimados

Os valores ajustados para os parâmetros epidemiológicos foram:

$
\beta = 0.514 \pm 0.008 \quad (1.65%)
$
$
\gamma = 0.396 \pm 0.008 \quad (2.04%)
$

As **incertezas relativas baixas** indicam que os parâmetros foram bem identificados pelos dados no período analisado.

Do ponto de vista epidemiológico:

* **β (taxa de contágio)** representa a intensidade das interações efetivas capazes de gerar novas infecções;
* **γ (taxa de recuperação)** está associada ao inverso do tempo médio de infecção.

A partir desses valores, pode-se estimar o **número básico de reprodução**:

$
R_0 = \frac{\beta}{\gamma} \approx \frac{0.514}{0.396} \approx 1.30
$

Esse valor é compatível com um cenário de **crescimento epidêmico sustentado**, porém moderado, coerente com a fase inicial da pandemia e com a adoção parcial de medidas de mitigação ao longo do período.

---

### 🔗 Correlação entre Parâmetros

O ajuste revelou uma **correlação extremamente alta** entre os parâmetros ( $\beta$ ) e ( $\gamma$ ):

$
\text{Corr}(\beta, \gamma) = 0.997
$

Essa correlação quase perfeita indica que os parâmetros **não são completamente independentes**, o que é típico em modelos compartimentais ajustados a **casos acumulados**. Em termos práticos, diferentes combinações de ( $\beta$ ) e ( $\gamma$ ) podem produzir trajetórias muito semelhantes para ( C(t) ), dificultando a identificação individual de cada parâmetro.

Esse resultado sugere que:

* o ajuste é fortemente sensível à **razão ( $\beta$ / $\gamma$ )** (isto é, ao ( $R_0$ ));
* a inclusão de dados adicionais, como casos ativos ou recuperados, poderia reduzir essa degenerescência;
* modelos com parâmetros variando no tempo poderiam capturar melhor mudanças estruturais na dinâmica da epidemia.

---

### 📌 Considerações Finais

Em síntese:

* o modelo SIRC apresentou **excelente desempenho descritivo** para os dados analisados;
* os parâmetros estimados são **estatisticamente bem definidos**, porém altamente correlacionados;
* o valor estimado de ( $R_0$ ) é consistente com a dinâmica observada no período inicial da pandemia;
* os resultados reforçam tanto a **utilidade** quanto as **limitações** de modelos compartimentais simples quando aplicados a dados reais agregados.

Esses achados destacam a importância de interpretar os parâmetros epidemiológicos **em conjunto com o contexto dos dados e das hipóteses do modelo**, evitando leituras puramente numéricas dos resultados.

